In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from google.colab import drive
import os
import zipfile
from tqdm import tqdm

# Mount Google Drive
drive.mount('/content/drive')

# Create a directory for saving results
save_path = '/content/drive/My Drive/CIFAR10_reconstruction'
os.makedirs(save_path, exist_ok=True)

# Define the Encoder-Decoder Architecture
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(16, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load CIFAR-10 datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                          download=True, transform=transform)

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss function, and optimizer
model = AutoEncoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for data in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images, _ = data
        images = images.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, images)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

# Generate and save reconstructed images
model.eval()
with torch.no_grad():
    for i, (images, _) in enumerate(test_loader):
        images = images.to(device)
        reconstructed = model(images)

        # Save each image in batch
        for j in range(images.size(0)):
            idx = i * batch_size + j
            if idx >= len(test_dataset):
                break

            # Convert to numpy array and save
            recon_img = reconstructed[j].cpu().numpy()
            save_file = os.path.join(save_path, f'image_{idx:03d}.npy')
            np.save(save_file, recon_img)

# Create zip file of all reconstructed images
zip_path = os.path.join(save_path, 'reconstructed_images.zip')
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for i in range(len(test_dataset)):
        file_name = f'image_{i:03d}.npy'
        file_path = os.path.join(save_path, file_name)
        zipf.write(file_path, file_name)

print("Processing complete! Files saved to Google Drive.")

# Calculate MSE for verification
def calculate_mse():
    total_mse = 0
    count = 0
    with torch.no_grad():
        for i, (images, _) in enumerate(test_loader):
            images = images.to(device)
            reconstructed = model(images)
            mse = criterion(reconstructed, images).item()
            total_mse += mse
            count += 1
    return total_mse / count

final_mse = calculate_mse()
print(f"Average MSE on test set: {final_mse:.4f}")


Mounted at /content/drive


100%|██████████| 170M/170M [00:04<00:00, 39.5MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Epoch 1/50: 100%|██████████| 391/391 [01:17<00:00,  5.07it/s]


Epoch [1/50], Average Loss: 0.0194


...

Epoch [48/50], Average Loss: 0.0026


Epoch 49/50: 100%|██████████| 391/391 [01:16<00:00,  5.10it/s]


Epoch [49/50], Average Loss: 0.0026


Epoch 50/50: 100%|██████████| 391/391 [01:17<00:00,  5.05it/s]


Epoch [50/50], Average Loss: 0.0025
Processing complete! Files saved to Google Drive.
Average MSE on test set: 0.0026
